In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 5.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=15ff768dd1cf0c0a0858ffae7795e1e3476fb14005228179f669572ac81f6089
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:

!mkdir convnext

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as ttf
from torchvision import models
import torchsummary
from torchsummary import summary
import os
import os.path as osp

from tqdm import tqdm
from PIL import Image
from sklearn.metrics import roc_auc_score
import numpy as np
import math


torch.cuda.empty_cache()

# Download Data

In [ ]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":" ","key":" "}') # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# # !kaggle competitions down!kaggle competitions download -c 11-785-s22-hw2p2-classification
!kaggle competitions download -c 11-785-s22-hw2p2-classification
!kaggle competitions download -c 11-785-s22-hw2p2-verification

!unzip -q 11-785-s22-hw2p2-classification.zip
!unzip -q 11-785-s22-hw2p2-verification.zip

!ls

 99% 2.34G/2.35G [00:09<00:00, 192MB/s]
100% 2.35G/2.35G [00:10<00:00, 253MB/s]
 95% 249M/263M [00:03<00:00, 65.0MB/s]
100% 263M/263M [00:03<00:00, 71.6MB/s]
11-785-s22-hw2p2-classification.zip   sample_data
11-785-s22-hw2p2-verification.zip     train_subset
classification			      verification
classification_sample_submission.csv  verification_sample_submission.csv
convnext


In [ ]:
# Hyperparameters

"""
The well-accepted SGD batch_size & lr combination for CNN classification is 256 batch size for 0.1 learning rate.
When changing batch size for SGD, follow the linear scaling rule - halving batch size -> halve learning rate, etc.
This is less theoretically supported for Adam, but in my experience, it's a decent ballpark estimate.
"""
batch_size = 256
lr = 0.1
epochs = 60

#  Network

In [ ]:
class InvertedResidualBlock(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels,
                 ):
        super().__init__() # Just have to do this for all nn.Module classes


        #Depthwise Convolution
        self.spatial_mixing = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=7, padding =3,
                    stride = 1, groups = in_channels, bias=False),
            nn.BatchNorm2d(in_channels),

        )

        # Expand Ratio is like 4, so hidden_dim >> in_channels
        hidden_dim = in_channels * 4

        #Pointwise Convolution
        self.feature_mixing = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, kernel_size=1, padding =0,
                    stride = 1, bias=False),
            nn.GELU(),
        )

        self.bottleneck_channels = nn.Sequential(
             nn.Conv2d(hidden_dim,out_channels,kernel_size=1,stride=1,padding=0,bias=False),
        )

    def forward(self, x):
        out = self.spatial_mixing(x)
        out = self.feature_mixing(out)
        out = self.bottleneck_channels(out)
        return x + out

class ConvNext(nn.Module):

    def __init__(self, num_classes= 7000):
        super().__init__()

        self.num_classes = num_classes

        """
        First couple of layers are special, just do them here.
        This is called the "stem". Usually, methods use it to downsample or twice.
        """
        self.stem = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=4, stride=4),
            nn.BatchNorm2d(96),
        )


        self.stage_cfgs = [
            # expand_ratio, channels, # blocks, stride of first block
            [4,  96, 3, 1],
            [4,  192, 3, 1],
            [4,  384, 9, 1],
            [4,  768, 3, 1],

        ]

        in_channels = 96
        layers = []


        #BLOCK TYPE 1 - 3 TIMES
        for i in range(3):
            layers.append(InvertedResidualBlock(
                in_channels=96,
                out_channels=96))

        layers.append(nn.BatchNorm2d(96))
        layers.append(nn.Conv2d(96,192,kernel_size=2,stride=2))

        #BLOCK TYPE 2 - 3 TIMES
        for i in range(3):
            layers.append(InvertedResidualBlock(
                in_channels=192,
                out_channels=192))

        layers.append(nn.BatchNorm2d(192))
        layers.append(nn.Conv2d(192,384,kernel_size=2,stride=2))

        #BLOCK TYPE 3 - 9 TIMES
        for i in range(9):
            layers.append(InvertedResidualBlock(
                in_channels=384,
                out_channels=384))

        layers.append(nn.BatchNorm2d(384))
        layers.append(nn.Conv2d(384,768,kernel_size=2,stride=2))

        #BLOCK TYPE 4 - 3 TIMES
        for i in range(3):
            layers.append(InvertedResidualBlock(
                in_channels=768,
                out_channels=768))



        self.layers = nn.Sequential(*layers)
        self.mid_cls_layer = nn.Sequential(

            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
        )
        self.final_cls_layer = nn.Sequential(nn.Linear(768,num_classes),)



    def forward(self, x,return_feats=False):
        out = self.stem(x)
        out = self.layers(out)

        feats = self.mid_cls_layer(out)

        out = self.final_cls_layer(feats)


        if return_feats:
            return feats
        else:
            return out


In [ ]:
"""
Transforms (data augmentation) is quite important for this task.
Go explore https://pytorch.org/vision/stable/transforms.html for more details
"""
DATA_DIR = "/content"
TRAIN_DIR = osp.join(DATA_DIR, "classification/classification/train") 
VAL_DIR = osp.join(DATA_DIR, "classification/classification/dev")
TEST_DIR = osp.join(DATA_DIR, "classification/classification/test")



train_transforms = [ttf.RandAugment(),
                    ttf.RandomHorizontalFlip(),
                    ttf.ColorJitter((0.8,1.2),(0.8,1.2),(0.8,1.2)),
                    ttf.ToTensor()]
val_transforms = [ttf.ToTensor()]

train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR,
                                                 transform=ttf.Compose(train_transforms))
val_dataset = torchvision.datasets.ImageFolder(VAL_DIR,
                                               transform=ttf.Compose(val_transforms))


train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, drop_last=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        drop_last=True, num_workers=1)

# Setup everything for training

In [ ]:
model = ConvNext()
# model.load_state_dict(torch.load('')) # Add modle location to resume training from a checkpoint
model.cuda()


num_trainable_parameters = 0
for p in model.parameters():
    num_trainable_parameters += p.numel()
print("Number of Params: {}".format(num_trainable_parameters))

# TODO: What criterion do we use for this task?
criterion = nn.CrossEntropyLoss(label_smoothing=0.25)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4,nesterov=True)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
# T_max is "how many times will i call scheduler.step() until it reaches 0 lr?"

#Using Mixed Precision
scaler = torch.cuda.amp.GradScaler()

Number of Params: 33155224


In [ ]:
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
       BatchNorm2d-2           [-1, 96, 56, 56]             192
            Conv2d-3           [-1, 96, 56, 56]           4,704
       BatchNorm2d-4           [-1, 96, 56, 56]             192
            Conv2d-5          [-1, 384, 56, 56]          36,864
              GELU-6          [-1, 384, 56, 56]               0
            Conv2d-7           [-1, 96, 56, 56]          36,864
InvertedResidualBlock-8           [-1, 96, 56, 56]               0
            Conv2d-9           [-1, 96, 56, 56]           4,704
      BatchNorm2d-10           [-1, 96, 56, 56]             192
           Conv2d-11          [-1, 384, 56, 56]          36,864
             GELU-12          [-1, 384, 56, 56]               0
           Conv2d-13           [-1, 96, 56, 56]          36,864
InvertedResidualBlock-14           [

# Let's train!

In [ ]:

PATH = '/content/convnext/'

In [13]:
for epoch in range(epochs):
    model.train()
    # Quality of life tip: leave=False and position=0 are needed to make tqdm usable in jupyter
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    num_correct = 0
    total_loss = 0

    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        # if torch.cuda.is_available():
        x = x.cuda()
        y = y.cuda()

        # Don't be surprised - we just wrap these two lines to make it work for FP16
        with torch.cuda.amp.autocast():     
            outputs = model(x)
            loss = criterion(outputs, y)

        # Update # correct & loss as we go
        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss += float(loss)

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        scheduler.step() # We told scheduler T_max that we'd call step() (len(train_loader) * epochs) many times.

        batch_bar.update() # Update tqdm bar
    batch_bar.close() # You need this to close the tqdm bar

    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        100 * num_correct / (len(train_loader) * batch_size),
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    

    # You can add validation per-epoch here if you would like
    if epoch%5 ==0:
      model.eval()
      batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
      num_correct = 0
      for i, (x, y) in enumerate(val_loader):
        x = x.cuda()
        y = y.cuda()
        with torch.no_grad():
          outputs = model(x)

        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)))

        batch_bar.update()
      batch_bar.close()
      print("Validation:"+str(epoch)+":"+"{:.04f}%".format(100 * num_correct / len(val_dataset)))


      torch.save(model.state_dict(), PATH+"epoch"+str(epoch)+".pt")
  



Epoch 1/60: Train Acc 0.0780%, Train Loss 8.7148, Learning Rate 0.0999


Validation:0:0.4571%


Epoch 2/60: Train Acc 1.3293%, Train Loss 8.0538, Learning Rate 0.0997


Epoch 3/60: Train Acc 7.7045%, Train Loss 7.2784, Learning Rate 0.0994


Epoch 4/60: Train Acc 22.4481%, Train Loss 6.4843, Learning Rate 0.0989


...

Validation:55:88.8057%


Epoch 57/60: Train Acc 99.9685%, Train Loss 2.9791, Learning Rate 0.0006


Epoch 58/60: Train Acc 99.9649%, Train Loss 2.9788, Learning Rate 0.0003


Epoch 59/60: Train Acc 99.9757%, Train Loss 2.9779, Learning Rate 0.0001


Epoch 60/60: Train Acc 99.9750%, Train Loss 2.9773, Learning Rate 0.0000


In [14]:
torch.save(model.state_dict(), PATH+"epoch"+str(epoch)+".pt")

# Classification Task: Validation

In [15]:
# !zip -r /outputs.zip /content/convent

In [16]:
# !unzip -q /content/outputs.zip

In [17]:

model.eval()
batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
num_correct = 0
for i, (x, y) in enumerate(val_loader):

    x = x.cuda()
    y = y.cuda()

    with torch.no_grad():
        outputs = model(x)

    num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
    batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)))

    batch_bar.update()
    
batch_bar.close()
print("Final_Validation: {:.04f}%".format(100 * num_correct / len(val_dataset)))

Final_Validation: 88.9629%


# Classification Task: Testing

In [18]:
class ClassificationTestSet(Dataset):
    # It's possible to load test set data using ImageFolder without making a custom class.
    # See if you can think it through!

    def __init__(self, data_dir, transforms):
        self.data_dir = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in data_dir
        self.img_paths = list(map(lambda fname: osp.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))

In [19]:
test_dataset = ClassificationTestSet(TEST_DIR, ttf.Compose(val_transforms))
print(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         drop_last=False, num_workers=1)


In [20]:
res = []
model.eval()
batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, position=0, leave=False, desc='Test')


for i, (x) in enumerate(test_loader):

    x = x.cuda()
    with torch.no_grad():
        outputs = model(x)
        pred_y = torch.argmax(outputs, axis=1)
        res.extend(pred_y.tolist())
    

    batch_bar.update()

batch_bar.close()

In [21]:

with open("/content/classification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    print(len(test_dataset))
    print(len(res))
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(6) + ".jpg", res[i]))
        
torch.save(model.state_dict(), "/content/Final_model_final.pth")

35000
35000


# Verification Task: Validation

There are 6K verification dev images, but 166K "pairs" for you to compare. So, it's much more efficient to compute the features for the 6K verification images, and just compare afterwards.

This will be done by creating a dictionary mapping the image file names to the features. Then, you'll use this dictionary to compute the similarities for each pair.

In [24]:
class VerificationDataset(Dataset):
    def __init__(self, data_dir, transforms):
        self.data_dir = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in data_dir
        self.img_paths = list(map(lambda fname: osp.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        # We return the image, as well as the path to that image (relative path)
        return self.transforms(Image.open(self.img_paths[idx])), osp.relpath(self.img_paths[idx], self.data_dir)

In [25]:

val_veri_dataset = VerificationDataset(osp.join(DATA_DIR, "verification/verification/dev"),
                                       ttf.Compose(val_transforms))
val_ver_loader = torch.utils.data.DataLoader(val_veri_dataset, batch_size=batch_size, 
                                             shuffle=False, num_workers=1)

In [50]:
model.eval()

feats_dict = dict()
for batch_idx, (imgs, path_names) in tqdm(enumerate(val_ver_loader), total=len(val_ver_loader), position=0, leave=False):
    imgs = imgs.cuda()

    with torch.no_grad():
        
        feats = model(imgs) 
        feats = nn.GELU()(feats)
        for i in range(len(feats)):
            address='dev/'+path_names[i]
            feats_dict[address] = feats[i]          

In [51]:
# # What does this dict look like?
z=feats_dict['dev/ab001b21a1.jpg']
x=feats_dict['dev/10246770ce.jpg']
nn.CosineSimilarity(dim=0)(z,x)

tensor(0.5595, device='cuda:0')

In [52]:
# We use cosine similarity between feature embeddings.
similarity_metric = nn.CosineSimilarity(dim=0)

val_veri_csv = osp.join(DATA_DIR, "verification/verification/verification_dev.csv")


# Now, loop through the csv and compare each pair, getting the similarity between them
pred_similarities = []
gt_similarities = []
for line in tqdm(open(val_veri_csv).read().splitlines()[1:], position=0, leave=False): # skip header
    img_path1, img_path2, gt = line.split(",")


    # TODO: Use the similarity metric
    # How to use these img_paths? What to do with the features?
    similarity = similarity_metric(feats_dict[img_path1],feats_dict[img_path2])

    pred_similarities.append(similarity)
    gt_similarities.append(int(gt))

pred_similarities = np.array(pred_similarities)
print(pred_similarities)
gt_similarities = np.array(gt_similarities)
print(gt_similarities)

print("AUC:", roc_auc_score(gt_similarities, pred_similarities))

[tensor(0.5595, device='cuda:0') tensor(0.1142, device='cuda:0')
 tensor(0.2032, device='cuda:0') ... tensor(0.2885, device='cuda:0')
 tensor(0.4706, device='cuda:0') tensor(0.7287, device='cuda:0')]
[1 0 0 ... 0 1 1]
AUC: 0.9649446129731225


# Verification Task: Testing

In [53]:
test_veri_dataset = VerificationDataset(osp.join(DATA_DIR, "verification/verification/test"),
                                        ttf.Compose(val_transforms))
test_ver_loader = torch.utils.data.DataLoader(test_veri_dataset, batch_size=batch_size, 
                                              shuffle=False, num_workers=1)

In [54]:
model.eval()

feats_final_dict = dict()
for batch_idx, (imgs, path_names) in tqdm(enumerate(test_ver_loader), total=len(test_ver_loader), position=0, leave=False):
    imgs = imgs.cuda()

    with torch.no_grad():
        feats = model(imgs)
        feats = nn.GELU()(feats) 
        for i in range(len(feats)):
            address='test/'+path_names[i]
            feats_final_dict[address] = feats[i]

In [55]:

val_veri_csv = osp.join(DATA_DIR, "verification/verification/verification_test.csv")


pred_similarities = []
for line in tqdm(open(val_veri_csv).read().splitlines()[1:], position=0, leave=False): # skip header
    img_path1, img_path2 = line.split(",")

    similarity = similarity_metric(feats_final_dict[img_path1],feats_final_dict[img_path2])
   
    pred_similarities.append(float(similarity))
    

pred_similarities = np.array(pred_similarities)

In [56]:
print(pred_similarities)

[0.43448931 0.60703105 0.53404409 ... 0.18287095 0.46324837 0.75750363]


In [57]:
with open("/content/verification_early_submission.csv", "w+") as f:
    f.write("id,match\n")
    for i in range(len(pred_similarities)):
        f.write("{},{}\n".format(i, pred_similarities[i]))

# Extras

In [35]:
# # If you keep re-initializing your model in Colab, can run out of GPU memory, need to restart.
# # These three lines can help that - run this before you re-initialize your model

# del model
# torch.cuda.empty_cache()
# !nvidia-smi